In [108]:
import pandas as pd
import numpy as np
import glob
import re
import datetime

In [3]:
def process_left(data):
    data["flag"] = np.where(data[1].apply(lambda x: "." in x), 1, 0)
    df1 = data[data["flag"] == 1]
    df2 = data[data["flag"] == 0]
    new_col = df2.iloc[:,0] + '0' + df2.iloc[:,1]
    df1 = df1[0]
    df_new = pd.concat([df1, new_col]).sort_index()
    return df_new

In [4]:
def process_left_73_96(data):
    data["flag"] = np.where(data[1].apply(lambda x: len(x) > 8), 0, 1)
    df1 = data[data["flag"] == 1]
    df2 = data[data["flag"] == 0]
    left_1 = df2.iloc[:,1].apply(lambda x:x[:1]).tolist()
    new_col = df2.iloc[:,0] + '0' + left_1
    df1 = df1[0]
    df_new = pd.concat([df1, new_col]).sort_index()
    return df_new

In [5]:
def extractLast4(x):
    elements = re.split('\s+', x.strip())
    return ",".join(elements[-4:])

In [6]:
def preprocessFile(path, types):
    list1 = []
    list2 = []
    with open(path, 'r') as f:
        for line in f:
            line1 = line.strip().split(".  ")
            list1.append(line1[0])
            list2.append(line1[-1])
    list2 = [extractLast4(x) for x in list2]
    list1 = pd.Series(list1)
    list2 = pd.Series(list2)
    data_right = pd.DataFrame(list2.str.split(',', expand = True))
    data_right.columns = ["CMI","PY","GM","CC"]
    data_right = data_right.drop(["PY","GM","CC"], axis = 1) 
    
    if types == 1:
        data_left = pd.DataFrame(list1.str.split('\s+', n = 1, expand = True))
        data_left = pd.DataFrame(process_left_73_96(data_left))
    if types == 2:
        data_left = pd.DataFrame(list1.str.split('\s+', n = 2, expand = True))
        data_left = pd.DataFrame(process_left(data_left))
    data_left.columns = ["id"]
    res = data_left.merge(data_right, how='left', left_index = True, right_index=True)
    return res

In [8]:
res = pd.DataFrame()
for filepath in glob.iglob(r'C:/Users/Chi Zhang/Desktop/CMI/Data/palmer9[789]*.txt'):
    res = pd.concat([res, preprocessFile(filepath, 2)])
for filepath in glob.iglob(r'C:/Users/Chi Zhang/Desktop/CMI/Data/palmer[01]*.txt'):
    res = pd.concat([res, preprocessFile(filepath, 2)])    
for filepath in glob.iglob(r'C:/Users/Chi Zhang/Desktop/CMI/Data/palmer[78]*.txt'):
    res = pd.concat([res, preprocessFile(filepath, 1)])   
for filepath in glob.iglob(r'C:/Users/Chi Zhang/Desktop/CMI/Data/palmer9[0123456]*.txt'):
    res = pd.concat([res, preprocessFile(filepath, 1)])     

In [9]:
res['len'] = res['id'].apply(len)
res = res[res['len'] <= 10].drop('len', axis = 1)

In [11]:
res.to_csv("CMI_73_18.csv", index=False)

In [165]:
FL_CMI = res[res['id'].str.match(r'^80')]

In [166]:
#Dic for all loctaions, (807 is manually added)

In [167]:
loc_dic = {
    "801" : "PENSACOLA REGIONAL AIRPORT",
    "802" : "TALLAHASSEE REGIONAL AIRPORT",
    "803" : "JACKSONVILLE INTERNATIONAL AIRPORT",
    "804" : "ORLANDO INTERNATIONAL AIRPORT",
    "805" : "TAMPA INTERNATIONAL AIRPORT",
    "806" : "MIAMI INTERNATIONAL AIRPORT",
    "807" : "Key West"
}

In [178]:
FL_CMI['loc_id'] = FL_CMI['id'].apply(lambda x: x[:3]).tolist()
FL_CMI['location'] = FL_CMI['loc_id'].apply(lambda x : loc_dic[x])

In [169]:
FL_CMI = FL_CMI.drop(['loc_id'], axis = 1)

In [171]:
FL_CMI['time'] = FL_CMI['id'].apply(lambda x: '19' + x[-4:] if len(x) == 7 else x[-6:]).tolist()

In [172]:
FL_CMI['more_52'] = FL_CMI['time'].apply(lambda x: 1 if int(x[-2:]) > 52 else 0)
FL_CMI = FL_CMI[FL_CMI['more_52'] == 0].drop('more_52', axis = 1)

In [173]:
FL_CMI['ymd_hms'] = FL_CMI['time'].apply(lambda x: datetime.datetime.strptime(x + '-1', "%Y%W-%w"))

In [174]:
FL_CMI = FL_CMI.drop(['time'], axis = 1)
FL_CMI = FL_CMI.drop(['id'], axis = 1)
FL_CMI = FL_CMI[["ymd_hms","location","CMI"]]

In [177]:
FL_CMI.to_csv("FL_CMI_73_18.csv", index=False)

In [175]:
FL_CMI

,ymd_hms,location,CMI
2132,1997-01-06,PENSACOLA REGIONAL AIRPORT,0.80
2133,1997-01-13,PENSACOLA REGIONAL AIRPORT,0.47
2134,1997-01-20,PENSACOLA REGIONAL AIRPORT,0.68
2135,1997-01-27,PENSACOLA REGIONAL AIRPORT,0.18
2136,1997-02-03,PENSACOLA REGIONAL AIRPORT,0.00
...,...,...,...
2491,1996-11-25,Key West,0.37
2492,1996-12-02,Key West,0.26
2493,1996-12-09,Key West,0.23
2494,1996-12-16,Key West,0.00
